In [ ]:
#All data 
import pandas as pd
import numpy as np
df = pd.read_csv('dfen2019.csv')
df=df.drop(columns=['label'])
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['date']=df['date'].astype('datetime64[ns]')
#df=df[['id_ticket','store','hour','product_code','product_name','department','category','brand','sales_qty','sales_value','price','date','client_id','date']]
df=df.loc[df['sales_value']>=0]
scores=pd.read_csv('scoresall.csv')
indscores=scores.groupby(['client_id','id_ticket','date','hedonic','utilitarian','both'])['sales_value'].sum().reset_index()
num=scores['client_id'].unique().tolist()
df=df.loc[df['client_id'].isin(num)]
df["date"] = pd.to_datetime(df["date"])

In [6]:
special=pd.read_csv('holiday.csv')
df["month_name"] = df["date"].dt.month_name()
months=df.month_name.unique().tolist()

In [7]:
#use store names
datastore=[[2,'Imbituba(2:01)',22,2589.31,-28.239597227964204, -48.66589713944407],
          [3,'Michel(3:05)',6,477.00,-28.65001837373967, -49.368464996422176],
          [4,'Urussanga(4:07)',10,1050.00,-28.493231753995776, -49.310786774692254],
          [5,'Garopaba(5:10)',18,1642.15,-28.004400040474923, -48.62232508648647],
          [6,'Orleans(6:11)',13,1335.08,-28.33407759699044, -49.28778066130743],
          [7,'Laguna(7:12)',16,1490.00,-28.45231475303656, -48.7880343261746],
          [9,'Tubarao(9:14)',10,1440.00,-28.466534159331573, -49.01635396491701],
           [15,'Pinheirinho(15:15)',7,880.78,-28.663274205161887, -49.39593081629356],
           [16,'Cocal(16:16)',5,375.56,-28.5308073872356, -49.32827471169928],
           [17,'Caravaggio(17:17)',5,500.00,-28.63054493198368, -49.47977160788221],
           [19,'Metropol(19:19)',5,500.00,-28.61679196637369, -49.43851570845118]]
dfstore=pd.DataFrame(datastore,columns=['store','branch','checkout','sales_area','Longitude','Latitude'])
store=dfstore['store'].tolist()
branch=dfstore['branch'].tolist()
df['branch']=df['store'].replace(store,branch)

In [8]:
import plotly as plt
import plotly.express as px
import json
from urllib.request import urlopen
import pandas as pd
with urlopen('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson') as response:
    Brazil = json.load(response) # Javascrip object notation 
state_id_map = {}
for feature in Brazil ['features']:
    feature['id'] = feature['properties']['name']
    state_id_map[feature['properties']['sigla']] = feature['id']
soybean = pd.read_csv('https://raw.githubusercontent.com/nayanemaia/Dataset_Soja/main/soja%20sidra.csv')
f = open('stanford-xw152bd5158-geojson.json',encoding="utf8")
Santa = json.load(f)
Santa_state = {}
names=[]
for feature in Santa ['features']:
    feature['id'] = feature['properties']['nome']
    names.append(feature['id'] )
    Santa_state[feature['properties']['nome']] = feature['id']
lst=dfstore.branch.tolist()
lstnew=['Imbituba','Criciúma ','Urussanga','Garopaba','Orleans','Laguna','Tubarão','Criciúma ','Cocal do Sul','Nova Veneza','Criciúma']
dfstore['loc']=dfstore['branch'].replace(lst,lstnew)
dfdate = df[['date', 'sales_value','branch','client_id']].groupby(['branch',pd.Grouper(key='date', freq='Y')]).agg({'sales_value':'sum', 
                             'client_id':'nunique'}).reset_index()
dfdate=dfdate.merge(dfstore[['branch','loc']],how='left',on='branch')

In [9]:
feature2019month = pd.read_csv('gender_cluster.csv')
feature2019month['child']=feature2019month['child'].replace(np.nan,0)
feature2019month['women']=feature2019month['women'].replace(np.nan,0)
feature2019month['men']=feature2019month['men'].replace(np.nan,0)
feature2019month=feature2019month.loc[feature2019month['client_id'].isin(num)]
feature2019month['month']=feature2019month['month'].replace(df.month.unique().tolist(),df.month_name.unique().tolist())


In [10]:
fem=feature2019month.loc[(feature2019month['women']==1)&(feature2019month['men']==0)&(feature2019month['child']==0)]
femlist=fem.client_id.unique().tolist()
femclust=fem.groupby(['client_id','Cluster'])['women'].sum().reset_index()
femtab = pd.pivot_table(femclust, values='women', index=['client_id'],columns=['Cluster'], aggfunc=np.sum, fill_value=0)
femtab=femtab.reset_index()
femtab['Max'] = femtab[['average','best','worst']].idxmax(axis=1)

In [11]:
dfsun=feature2019month.groupby(['month','Cluster'])['client_id'].nunique().reset_index()
tablenew = pd.pivot_table(dfsun, values='client_id', index=['Cluster'],columns=['month'], aggfunc=np.sum, fill_value=0)
tablenew=tablenew.reset_index()
df3=tablenew[months]
tablenew22 = df3.div(df3.sum(axis=0), axis=1)
tablenew22=tablenew22*100
df3 = df3.apply(pd.to_numeric, errors='coerce')
sum_3 = df3.sum(axis = 0, skipna = True)
import seaborn as sns
a=sns.color_palette('hls',3)
a=a.as_hex()
data = {"best": tablenew22.iloc[1].tolist(),
        "average"    : tablenew22.iloc[0].tolist(),
        "worst"     : tablenew22.iloc[2].tolist()
        }
colors= {"best": a[1],
         "average"    : a[2],
         "worst"     : a[0],
        }
widths  = sum_3
x_width = sum_3.sum()

In [12]:
table = pd.read_csv('general_table.csv')
dep=df.groupby(['department','category','product_name','month'])['sales_value'].sum().reset_index()
dep=dep.merge(table[['product_name','month','Cluster_names']],how='left',on=['product_name','month'])
dfdep=dep.groupby(['department','month','Cluster_names'])['product_name'].nunique().reset_index()
deptab = pd.pivot_table(dfdep, values='product_name', index=['department','Cluster_names'],columns=['month'], aggfunc=np.sum, fill_value=0)
deptab=deptab.reset_index()
deptabnew=pd.DataFrame()
for i in range(0,len(deptab),3):
    neww=deptab.iloc[i:i+3,2:].div(deptab.iloc[i:i+3,2:].sum(axis=0), axis=1)
    deptabnew=deptabnew.append(neww,ignore_index=False)
deptabnew=deptabnew*100
colst=[]
for col in deptab:
    colst.append(col)
for k in range(2,len(colst)):
    deptab[colst[k]]=deptabnew[colst[k]]
deptab=deptab.replace(np.nan,0)
lstnum=list(range(1,13))
res = { lstnum[i]:months[i] for i in range(len(months))}
deptab = deptab.rename(columns=res)


In [20]:
from jupyter_dash import JupyterDash     # pip install dash           
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input
import dash_table
from dash_extensions import Lottie       # pip install dash-extensions
import dash_bootstrap_components as dbc  # pip install dash-bootstrap-components
import plotly.express as px 
import plotly.graph_objects as go# pip install plotly
import pandas as pd                      # pip install pandas
from datetime import date
import calendar
         # pip install wordcloud

url_customer = "https://assets10.lottiefiles.com/private_files/lf30_eaaSTk.json"
url_stores = "https://assets4.lottiefiles.com/packages/lf20_wn85it4b.json"
url_category = 'https://assets7.lottiefiles.com/packages/lf20_zzbz9na6.json'
url_buckets = "https://assets10.lottiefiles.com/private_files/lf30_rgsilsi9.json"
url_sales = "https://assets6.lottiefiles.com/private_files/lf30_khwfxgwr.json"
url_transaction = "https://assets4.lottiefiles.com/packages/lf20_awucen2m.json"
options = dict(loop=True, autoplay=True, rendererSettings=dict(preserveAspectRatio='xMidYMid slice'))
url_female = 'https://assets3.lottiefiles.com/packages/lf20_rkd7ebny.json'
import base64
test_png = 'logo-mobile.png'
test_base64 = base64.b64encode(open(test_png, 'rb').read()).decode('ascii')
# Bootstrap themes by Ann: https://hellodash.pythonanywhere.com/theme_explorer
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LUX])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.Card([
                 html.Img(src='data:image/png;base64,{}'.format(test_base64)),
            ],className='mb-2'),
            dbc.Card([
                dbc.CardBody([
                ])
            ]),
        ], width=2),
         dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.DatePickerSingle(
                        id='my-date-picker-start',
                        date=date(2019, 1, 1),
                        className='ml-5'
                    ),
                    dcc.DatePickerSingle(
                        id='my-date-picker-end',
                        date=date(2019, 12, 31),
                        className='mb-2 ml-2'
                    ),
                ])
            ], color="info"),
        ], width=10),
    ],className='mb-2 mt-2'),
   dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="32%", height="32%", url=url_customer)),
                dbc.CardBody([
                    html.H6('Customers'),
                    html.H2(id='content-customer', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="45%", height="45%", url=url_stores)),
                dbc.CardBody([
                    html.H6('Stores'),
                    html.H2(id='content-stores', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="45%", height="45%", url=url_category)),
                dbc.CardBody([
                    html.H6('Category'),
                    html.H2(id='content-category', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="45%", height="45%", url=url_buckets)),
                dbc.CardBody([
                    html.H6('Buckets'),
                    html.H2(id='content-buckets', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="45%", height="45%", url=url_sales)),
                dbc.CardBody([
                    html.H6('Sales'),
                    html.H2(id='content-sales', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options, width="65%", height="65%", url=url_transaction)),
                dbc.CardBody([
                    html.H6('Transactions'),
                    html.H2(id='content-transaction', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
         
    ],className='mb-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.P("Average Sales of Stores"),
                    dcc.RadioItems(
                        id='radio_type', 
                        options=[{'label': 'Weekly', 'value': 'W'},
                                {'label': 'Montly', 'value': 'M'},
                                {'label': 'Daily', 'value': 'D'}],
                        value='W',
                        labelStyle={'display': 'inline-block'}
                    ),
                    dcc.Graph(id='line-chart', figure={}, config={'displayModeBar': False}),
                ])
            ]),
        ], width=8),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.P("Monthly Sales of Stores"),
                    dcc.Dropdown(
                        id='month', 
                        options=[{'value': x, 'label': x} 
                                 for x in months],
                        value=months[0]
                    ),
                    dcc.Graph(id='map-chart', figure={}),
                ])
            ]),
        ], width=4),
    ],className='mb-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.P("Monthly Customer Segment Percentages"),
                    dcc.Graph(id='percentage-cluster', figure={}),
                ])
            ]),
        ], width=6),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.P("Hed/UT percentage of department monthly"),
                    dcc.Dropdown(
                        id='month-dep', 
                        options=[{'value': x, 'label': x} 
                                 for x in months],
                        value=months[0]
                    ),
                    dcc.Graph(id='dep-cluster', figure={}),
                ])
            ]),
        ], width=6),
    ],className='mb-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dbc.CardHeader(Lottie(options=options, width="35%", height="35%", url=url_female))
                    ]),
            ],className='mb-2'),
            
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Customer Segment"),
                dbc.CardBody([
                    dcc.RadioItems(
                        id='radio_cluster', 
                        options=[{'label': 'Best', 'value': 'best'},
                                {'label': 'Average', 'value': 'average'},
                                {'label': 'Worst', 'value': 'worst'}],
                        value='best',
                        labelStyle={'display': 'inline-block'},
                        className='ml-5'
                    ),
                    
                ])
            ], color="info",inverse=True),
        ], width=4),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Bucket Scores"),
                dbc.CardBody([
                    dcc.RadioItems(
                        id='radio_hed', 
                        options=[{'label': 'Hedonic', 'value': 'hedonic'},
                                {'label': 'Utilitarian', 'value': 'utilitarian'},
                                {'label': 'Both', 'value': 'both'}],
                        value='hedonic',
                        labelStyle={'display': 'inline-block'}
                    ),
                    
                ])
            ], color="info",inverse=True),
        ], width=4),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Additionals"),
                dbc.CardBody([
                    dcc.RadioItems(
                        id='radio_add', 
                        options=[{'label': 'Cluster', 'value': 'cluster'},
                                {'label': 'Special Days', 'value': 'special'}],
                        value='cluster',
                        labelStyle={'display': 'inline-block'}
                    ),
                    
                ])
            ], color="info", inverse=True),
        ], width=2),
    ],className='mb-2 mt-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.P("Scores"),
                    dcc.Dropdown(id="females-dropdown",value=242417),
                    dcc.Graph(id='scores', figure={}),
                ])
            ]),
        ], width=12),
        
    ],className='mb-2'),
    
    dbc.Row([
       dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dash_table.DataTable(id='datatable',
                            columns=[
                                {"name": i, "id": i} for i in ['date','category','brand','product_name']]),
                ])
            ]),
        ], width=12),
        
    ],className='mb-2'),
], fluid=True)
def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.2f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

# Updating the 5 number cards
@app.callback(
    Output('content-customer','children'),
    Output('content-stores','children'),
    Output('content-category','children'),
    Output('content-buckets','children'),
    Output('content-sales','children'),
    Output('content-transaction','children'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date')
)
def update_small_cards(start_date, end_date):
    dff_c = df[(df['date']>=start_date) & (df['date']<=end_date)]
    customer=dff_c['client_id'].nunique()
    stores=dff_c['store'].nunique()
    category=dff_c['category'].nunique()
    buckets=dff_c['id_ticket'].nunique()
    sales=dff_c['sales_value'].sum()
    transaction=len(dff_c)
    return human_format(customer),stores,category,human_format(buckets),human_format(sales),human_format(transaction)
@app.callback(
    Output('line-chart','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('radio_type','value')
)
def update_line(start_date, end_date,value):
    timeslot={'W':'weekly','D':'daily','M':'monthly'}
    dff_c = df[(df['date']>=start_date) & (df['date']<=end_date)]
    dfdate = dff_c[['date', 'sales_value','branch','client_id']].groupby(['branch',pd.Grouper(key='date', freq=value)]).agg({'sales_value':'mean', 
                             'client_id':'nunique'}).reset_index()
    #dfdate['month']=pd.DatetimeIndex(dfdate['date']).month
    dfdate['week']=pd.DatetimeIndex(dfdate['date']).week
    #dfdate['day']=pd.DatetimeIndex(dfdate['date']).day
    fig = px.line(dfdate, x="date", y='sales_value',color='branch',
                  hover_data=['sales_value','client_id'])
    fig.update_layout(paper_bgcolor='#fff',plot_bgcolor='#fff')

    return fig

# Map ************************************************************
@app.callback(
    Output('map-chart','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('month','value')
)
def update_bar(start_date, end_date,value):
    dff_c = df[(df['date']>=start_date) & (df['date']<=end_date)&(df['month_name']==value)]
    dfdate = dff_c[['date', 'sales_value','branch','client_id']].groupby(['branch',pd.Grouper(key='date', freq='W')]).agg({'sales_value':'mean', 
                             'client_id':'nunique'}).reset_index()
    
   
    dfx=dfdate.merge(dfstore[['branch','loc']],how='left',on='branch')
    fig = px.choropleth(
     dfx, #soybean database
     locations = 'loc', #define the limits on the map/geography
     geojson = Santa, #shape information
     color = 'sales_value', #defining the color of the scale through the database
     hover_name = 'branch', #the information in the box
     hover_data =["sales_value",'loc'],
     #animation_frame = 'week' #creating the application based on the year
    )
    fig.update_geos(fitbounds = 'locations', visible = False)

    return fig
# Marimekko chart ************************************************************
@app.callback(
    Output('percentage-cluster','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date')
)
def update_mekko(start_date, end_date):
    fig = go.Figure()
    for key in data:
        fig.add_trace(go.Bar(name = key,
                             y = data[key],
                             x = np.cumsum(widths) - widths,
                             width = widths, offset = 0,
                             marker = {'color' : colors[key]},
                             customdata = data[key],
                             texttemplate = "%{y:.2f}",
                             textposition = "inside",
                             textangle = 0,textfont_color = "white",))
    fig.update_layout(title_text ="",
                      barmode = "stack")
    fig.update_xaxes(title_text='',
                     range = [0, x_width],  
                     title_font_size = 30,                   
                     tickvals = np.cumsum(widths)-widths/2,
                     ticktext =["%s"% (l) for l 
                                in months],
                     tickfont = dict(family='Calibri', 
                                    color='black', size=15))
    fig.update_yaxes(title_text = 'Percentage (%)',
                     range = [0,100],
                     title_font=dict(size=25,
                                     family='Verdana', color='darkred')
                    )

    return fig

# Department scores ************************************************************
@app.callback(
    Output('dep-cluster','figure'),
    Input('month-dep','value')
)
def update_dep(value):
    deneme=deptab[['department','Cluster_names',value]]
    test = pd.pivot_table(deneme, values=value, index=['department'],columns=['Cluster_names'], aggfunc=np.sum, fill_value=0)
    test=test.reset_index()
    fig = px.scatter(test, x="hedonic", y="utilitarian", color="department",hover_data=['hedonic','utilitarian','both','department'])

    return fig

# HED/UT scores ************************************************************
@app.callback(
    Output("females-dropdown", "options"),
    Input('radio_cluster','value')
)
def update_options(cluster_val):
    df_c=femtab.loc[femtab['Max']==cluster_val]
    result=[{'label': i, 'value': j} for i, j in zip(df_c['client_id'].unique(), df_c['client_id'].unique())]

    return result
@app.callback(
    Output('scores','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('radio_hed','value'),
    Input('radio_cluster','value'),
    Input('radio_add','value'),
    Input('females-dropdown','value')
)
def update_scores_fem(start_date, end_date,hed_val,cluster_val,add_val,idx):
    dff_c=scores.loc[scores['client_id']==idx]
    dff_c2 = dff_c[(dff_c['date']>=start_date) & (dff_c['date']<=end_date)]
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        name=hed_val,
        mode="markers+lines", x=dff_c2["date"], y=dff_c2[hed_val],

    ))
    if add_val=='special':
        for i in special.date.tolist():
            fig.add_vline(x=i, line_dash="dot")
    else:
        besty=dff_c2.loc[dff_c2['Customer_cluster']=='best']
        besty['val']=1
        fig.add_trace(go.Scatter(
            x=besty["date"],
            y=besty['val'],
           fill='tozeroy',
            name='best',
            mode='none',
            fillcolor='rgba(87, 219, 95, 0.3)'
        ))
        average=dff_c2.loc[dff_c2['Customer_cluster']=='average']
        average['val']=1
        fig.add_trace(go.Scatter(
            x=average["date"],
            y=average['val'],
           fill='tozeroy',
            name='average',
            mode='none',
            fillcolor='rgba(95, 87, 219, 0.3)'
        ))
        worst=dff_c2.loc[dff_c2['Customer_cluster']=='worst']
        worst['val']=1
        fig.add_trace(go.Scatter(
            x=worst["date"],
            y=worst['val'],
           fill='tozeroy',
            name='worst',
            mode='none',
            fillcolor='rgba(219, 95, 87, 0.3)'
        ))
    fig.update_xaxes(showgrid=False)

    fig.update_xaxes(dtick="M1",tickformat="%b\n%Y")
    fig.update_layout(paper_bgcolor='#fff',plot_bgcolor='#fff')

    return fig
@app.callback(
    Output('datatable', 'data'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
    Input('radio_hed','value'),
    Input('radio_cluster','value'),
    Input('radio_add','value'),
    Input('females-dropdown','value')
)
def update_datatable(start_date, end_date,hed_val,cluster_val,add_val,idx):
    dff_c=scores.loc[scores['client_id']==idx]
    dff_c2 = dff_c[(dff_c['date']>=start_date) & (dff_c['date']<=end_date)&(dff_c['Cluster']==hed_val)]
    dff_c3=dff_c2.nlargest(10, hed_val)
    dff=dff_c3[['date','category','brand','product_name']]
    return dff.to_dict('records')
app.run_server(mode='jupyterlab', port=8001)

<ipython-input-20-9a290635755a>:410: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-20-9a290635755a>:430: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-20-9a290635755a>:420: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

